In [ ]:
import os
import pandas as pd
from PIL import Image
import torch

from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

#### create path to my data on pc

In [ ]:
train_path = r"C:\Users\1\Desktop\University_semeter_3\ZNEUS\archive\ASL_Dataset\Train"

#### create csv file with image pases and labels

In [ ]:
data = []

for folder in os.listdir(train_path):
    folder_path = os.path.join(train_path, folder)

    if os.path.isdir(folder_path):
        for image in os.listdir(folder_path):
            if image.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, image)
                data.append({'path': image_path, 'label': folder})

# Створюємо DataFrame
df = pd.DataFrame(data)

# Зберігаємо в CSV
df.to_csv('dataset.csv', index=False)
print(f" {len(df)} rows")
print(f"Clases: {df['label'].unique()}")


In [ ]:
ds = pd.read_csv('dataset.csv')
ds.head()


In [ ]:
ds.info()

In [ ]:
ds.duplicated().sum()

In [ ]:
ds = ds.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
ds_answer =ds.iloc[:,-1:]
ds_answer.head()

In [ ]:
ds_train = ds.iloc[:,:-1]
ds_train.head()

In [ ]:
for i in range(3):
    img = Image.open(ds_train.iloc[i,0])
    img.show()




In [ ]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.ToTensor()
])

img_path = df.iloc[0]['path']
img = Image.open(img_path)

img_tensor = transform(img)

print(img_tensor.shape)

In [ ]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,12,3,1)
conv3 = nn.Conv2d(12,18,3,1)

In [ ]:
x = F.relu(conv1(img_tensor.unsqueeze(0)))
x.shape

In [ ]:
x = F.max_pool2d(x,2,2)
x.shape

In [81]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,12,3,1)
        self.conv3 = nn.Conv2d(12,18,3,1)
        self.fc1 = nn.Linear(18*12*12,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,28)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,18*12*12)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return F.log_softmax(x, dim=1)

In [82]:
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(12, 18, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2592, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=28, bias=True)
)